In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re

df=pd.read_csv('Tweets_hashtags_2020_jan_feb.csv',encoding='unicode_escape')
print('data loaded')
df=df[df['Screen Name']!='MichaeLawbzpc\n']
tweets=list(df['Tweets'])

# Import the Universal Sentence Encoder's TF Hub module
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
embed = hub.Module(module_url)

In [ ]:
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  tweet_embeddings = session.run(embed(tweets))

In [ ]:
tweet_embeddings.shape

## Tweet vectors using Universal Sentence Embeddings

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

for i in np.arange(1,1.2,0.1):
    
    clustering = DBSCAN(eps=i, min_samples=100,n_jobs=-1).fit(tweet_embeddings)
    print('no. of clusters',len(pd.Series(clustering.labels_).unique()),' eps:-',i)

In [ ]:
i=0.9
clustering = DBSCAN(eps=i, min_samples=100,n_jobs=-1).fit(tweet_embeddings)
print('no. of clusters',len(pd.Series(clustering.labels_).unique()),' eps:-',i)

In [ ]:
no_clusters=len(pd.Series(clustering.labels_).value_counts())

df['clusters_embeddings']=clustering.labels_

# Word clouds for each cluster

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

for c in range(-1,no_clusters-1):
    print('Cluster No.',c)
    hts=list(df[df['clusters_embeddings']==c]['Hashtags'])

    hashes=[]
    for ht in  hts:
        for h in eval(ht):
            hashes.append(h.strip())

    string_hash=' '.join(hashes)

    hash_values=pd.Series(hashes).value_counts()

    hval=hash_values.reset_index()

    #wordcloud plot
    d = {}
    for a, x in hval.values:
        d[a] = x

    wordcloud = WordCloud(max_font_size=40)
    wordcloud.generate_from_frequencies(frequencies=d)
    plt.figure(figsize=(100,100))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.show()
    plt.savefig('Cluster_jan_feb'+str(c))

In [ ]:
df[df['clusters_embeddings']==1]

In [ ]:
set(list(df['clusters_embeddings']))


In [ ]:
#Cluster Analysis
#Cluster -1 :-Outliers by algo
#Cluster 0:- General litigation
#Cluster 1:- Week day tweets
#Cluster 2:- whistleblowers
#Cluster 3:- boxing
#Cluster 4:- health related(teen Vaping,e-cigarette,Big Tobacco)
#Cluster 5:- Spanish,french..tweets(non-english)
#Cluster 6:- Injury, Accidents

In [ ]:
df